## Retrieve data

In [1]:
! curl "https://api.mockaroo.com/api/2a2d8dd0?count=1000&key=f6504dd0" > "Employees.csv"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 94853    0 94853    0     0  66379      0 --:--:--  0:00:01 --:--:-- 66377


## Setup Employee Database

In [2]:
# Define SQL schema for creating the employees table with specified fields
employees_schema = """
CREATE TABLE IF NOT EXISTS employees (
    employee_id INT PRIMARY KEY,
    first_name VARCHAR(50),
    last_name VARCHAR(50),
    age INT,
    email VARCHAR(100),
    gender VARCHAR(10),
    job_title VARCHAR(50),
    department VARCHAR(50),
    salary DECIMAL(10, 2),
    hire_date DATE
);
"""

In [3]:
import sqlite3
import pandas as pd
import os

In [4]:
# --- Database setup ---
db_name_employees = 'employees.db'
conn_employees = None  # Initialize connection to None

# Remove existing database if it exists
if os.path.exists(db_name_employees):
    os.remove(db_name_employees)
    print(f"Removed existing database '{db_name_employees}'.")

try:
    # Establish a connection to the SQLite database
    conn_employees = sqlite3.connect(db_name_employees)
    cursor_employees = conn_employees.cursor()
    print(f"Database '{db_name_employees}' created and connected successfully. ✅")

    # Create the employees table
    cursor_employees.execute(employees_schema)
    print("Table 'employees' created successfully.")

    # --- Load data from CSV file into the table using pandas ---
    csv_file_employees = '/content/Employees.csv'
    table_name_employees = 'employees'

    if os.path.exists(csv_file_employees):
        print(f"\nProcessing '{csv_file_employees}' for table '{table_name_employees}'...")

        # Read the CSV file into a pandas DataFrame
        df_employees = pd.read_csv(csv_file_employees)

        # Define expected schema and data types for employees based on the created schema
        employee_column_data_types = {
            'employee_id': 'int64',
            'first_name': 'object',
            'last_name': 'object',
            'age': 'int64',
            'email': 'object',
            'gender': 'object',
            'job_title': 'object',
            'department': 'object',
            'salary': 'float64',
            'hire_date': 'datetime64[ns]'
        }
        expected_cols_employees = list(employee_column_data_types.keys())

        # Handle missing/extra columns
        df_employees = df_employees[df_employees.columns.intersection(expected_cols_employees)]
        for col in expected_cols_employees:
            if col not in df_employees.columns:
                df_employees[col] = None

        # Reorder columns to match the defined schema exactly
        df_employees = df_employees[expected_cols_employees]

        # Enforce data types
        for col, dtype in employee_column_data_types.items():
            if 'datetime' in dtype:
                df_employees[col] = pd.to_datetime(df_employees[col], errors='coerce')
            else:
                try:
                    df_employees[col] = df_employees[col].astype(dtype)
                except (ValueError, TypeError) as e:
                    print(f"  - Warning: Could not convert column '{col}' to {dtype}. Error: {e}. Leaving as is.")


        # Use the to_sql method to insert the cleaned DataFrame
        df_employees.to_sql(table_name_employees, conn_employees, if_exists='append', index=False)
        print(f"  -> Data from '{csv_file_employees}' loaded into '{table_name_employees}' table successfully.")
    else:
        print(f"Warning: '{csv_file_employees}' not found. Skipping data load for '{table_name_employees}'.")

    # Commit the changes to the database
    conn_employees.commit()
    print("\nData committed to the database successfully. 🎉")

except sqlite3.Error as e:
    print(f"Database error: {e}")
except pd.errors.EmptyDataError as e:
    print(f"Pandas error: {e}. The Employees.csv file might be empty.")
except KeyError as e:
    print(f"Schema definition error: A column is missing from the employee_column_data_types dictionary: {e}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")
finally:
    # Close the connection if it was established
    if conn_employees:
        conn_employees.close()
        print("Database connection closed.")

Database 'employees.db' created and connected successfully. ✅
Table 'employees' created successfully.

Processing '/content/Employees.csv' for table 'employees'...
  -> Data from '/content/Employees.csv' loaded into 'employees' table successfully.

Data committed to the database successfully. 🎉
Database connection closed.


In [5]:
# Install the necessary library
!pip install google-genai

### Import required modules

In [6]:
from google import genai
from google.colab import userdata

In [7]:
genai_client = genai.Client(api_key=userdata.get('GOOGLE_API_KEY'))

## Prompt Engineering

In [ ]:
import json
def get_sql_query(genai_client, prompt, user_query):
  contents = f"""
  {prompt}

  Here's the user query in english you need to work on:
  {user_query}
  """
  response = genai_client.models.generate_content(model='gemini-2.5-flash', contents=contents)
  # print(response)

  # Access the usage_metadata attribute
  usage_metadata = response.usage_metadata

  # Print the different token counts
  print(f"Input Token Count: {usage_metadata.prompt_token_count}")
  print(f"Thoughts Token Count: {response.usage_metadata.thoughts_token_count}")
  print(f"Output Token Count: {usage_metadata.candidates_token_count}")
  print(f"Total Token Count: {usage_metadata.total_token_count}")

  output = json.loads(response.text.replace('```json', '').replace('```', ''))

  return output

In [43]:
prompt = """
### ROLE

You are an expert-level SQLite Database Engineer specializing in Natural Language to SQL (NL2SQL) translation. Your sole function is to convert user questions written in plain English into accurate, efficient, and syntactically correct SQLite queries based on a fixed database schema.

-----

### CONTEXT

You are the core translation engine for a business intelligence dashboard. This tool allows non-technical employees to query the company's employee database using natural language. The database dialect is always SQLite. Your responses will be executed directly on the database.

The database consists of the following table:

**employees table:**

CREATE TABLE employees (
    employee_id INT PRIMARY KEY,
    first_name VARCHAR(50),
    last_name VARCHAR(50),
    age INT,
    email VARCHAR(100),
    gender VARCHAR(10),
    job_title VARCHAR(50),
    department VARCHAR(50),
    salary DECIMAL(10, 2),
    hire_date DATE
);

-----

### TASK

Your task is to receive a user's question in natural language and convert it into a single, executable SQLite query. Follow these steps meticulously:

1. Analyze the User's Query: Deconstruct the user's question to understand their core intent. Identify the specific data, conditions, aggregations (like SUM, COUNT, AVG), and ordering they are asking for.
2. Map to the Schema: Map the entities from the user's query to the appropriate columns in the employees table.
3. Construct the SQLite Query: Write a clean and efficient SELECT statement that is syntactically correct for SQLite. Ensure all column names are accurate.
4. Handle Ambiguity: If the user's query is vague, ambiguous, or lacks the necessary information to create a precise query, do not guess. Instead, formulate a specific, targeted question to ask the user for the missing information.

-----

### CONSTRAINTS

* Read-Only Operations: You must ONLY generate SELECT queries. Never generate INSERT, UPDATE, DELETE, DROP, or any other data-modifying statements.
* Adhere Strictly to Schema: Only use the columns defined in the context. Do not invent or assume the existence of any other columns.
* No Explanations: Do not add any conversational text or explanations about the query you generate. Your output must strictly follow the specified format.
* Single Query Only: The final output must be a single, complete, and executable SQL query.
* Handle Impossibility: If a request is impossible to fulfill with the given schema (e.g., "Show me all projects"), state clearly that the request cannot be completed and briefly explain why.

-----

### OUTPUT FORMAT

Your final response must be a single JSON object with two keys:

1. "status": A string with one of three possible values: "success", "clarification_needed", or "error".
2. "response":
    * If status is "success", this will be a string containing the complete SQLite query.
    * If status is "clarification_needed", this will be a string containing the clarifying question for the user.
    * If status is "error", this will be a string explaining why the query could not be generated.
"""

In [44]:
import sqlite3
import pandas as pd

def execute_query(query, db_name='employees.db'):

    conn = None
    try:
        # Connect to the database
        conn = sqlite3.connect(db_name)
        cursor = conn.cursor()

        # Execute the query
        print(f"\nExecuting query on '{db_name}':\n{query}")
        cursor.execute(query)

        # Fetch all results
        results = cursor.fetchall()

        # Get column names from the cursor description
        columns = [description[0] for description in cursor.description]

        # Format results as a dataframe for easier use
        results_as_dict = [dict(zip(columns, row)) for row in results]
        results_df = pd.DataFrame(results_as_dict)

        print("Query executed successfully.")
        return results_df

    except sqlite3.Error as e:
        print(f"Database error executing query: {e}")
        return None
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return None
    finally:
        if conn:
            conn.close()

In [45]:
def text2sql(genai_client, prompt, user_query):
  output = get_sql_query(genai_client, prompt, user_query)
  if output['status'] == 'success':
    results = execute_query(output['response'])
    return results
  return output

## Query Employee Database

In [49]:
# Example query for the employee database
employee_query_result = text2sql(genai_client, prompt, "Show me the number of employees in each department")
display(employee_query_result)

Input Token Count: 724
Thoughts Token Count: 38
Output Token Count: 43
Total Token Count: 805

Executing query on 'employees.db':
SELECT department, COUNT(employee_id) AS number_of_employees FROM employees GROUP BY department;
Query executed successfully.


,department,number_of_employees
0,HR,243
1,IT,268
2,Marketing,244
3,Sales,245


In [50]:
#Get the average salary for each department
average_salary_by_department = text2sql(genai_client, prompt, "What is the average salary for each department?")
print("\nAverage Salary by Department:")
display(average_salary_by_department)



Input Token Count: 724
Thoughts Token Count: 42
Output Token Count: 39
Total Token Count: 805

Executing query on 'employees.db':
SELECT department, AVG(salary) AS average_salary FROM employees GROUP BY department;
Query executed successfully.

Average Salary by Department:


,department,average_salary
0,HR,66732.463004
1,IT,64164.923097
2,Marketing,64958.516189
3,Sales,63364.734653


In [51]:
# Find employees older than 50
employees_older_than_50 = text2sql(genai_client, prompt, "Show me the first name, last name, and age of employees older than 50")
print("\nEmployees Older than 50:")
display(employees_older_than_50)



Input Token Count: 733
Thoughts Token Count: 53
Output Token Count: 41
Total Token Count: 827

Executing query on 'employees.db':
SELECT first_name, last_name, age FROM employees WHERE age > 50;
Query executed successfully.

Employees Older than 50:


,first_name,last_name,age
0,Norrie,Strowther,55
1,Guendolen,MacAiline,64
2,Elliot,Holdworth,61
3,Cullin,Le Moucheux,64
4,Denys,Obern,51
...,...,...,...
297,Farr,Semrad,54
298,Saundra,Sizzey,57
299,Georges,Siley,63
300,Yehudi,Spellacey,54


In [53]:
# Count the number of employees hired in 2021
employees_hired_in_2021 = text2sql(genai_client, prompt, "How many employees were hired in 2021?")
print("\nNumber of Employees Hired in 2021:")
display(employees_hired_in_2021)


Input Token Count: 727
Thoughts Token Count: 137
Output Token Count: 45
Total Token Count: 909

Executing query on 'employees.db':
SELECT COUNT(*) FROM employees WHERE STRFTIME('%Y', hire_date) = '2021';
Query executed successfully.

Number of Employees Hired in 2021:


,COUNT(*)
0,84


In [54]:

# List the job titles in the IT department
it_job_titles = text2sql(genai_client, prompt, "List all the job titles in the IT department")
print("\nJob Titles in IT Department:")
display(it_job_titles)

Input Token Count: 724
Thoughts Token Count: 63
Output Token Count: 35
Total Token Count: 822

Executing query on 'employees.db':
SELECT DISTINCT job_title FROM employees WHERE department = 'IT';
Query executed successfully.

Job Titles in IT Department:


,job_title
0,Operator
1,Physical Therapy Assistant
2,Financial Analyst
3,Clinical Specialist
4,Compensation Analyst
...,...
119,Chief Design Engineer
120,Research Assistant II
121,Payment Adjustment Coordinator
122,Administrative Assistant III


# Conclusion

This notebook successfully demonstrates the creation of a Text-to-SQL system for an employee database. We have covered the following key aspects:

1.  **Dataset Creation:** A synthetic employee dataset with relevant fields was generated using Mockaroo.
2.  **Database Setup and Data Loading:** An SQLite database (`employees.db`) was created, and the employee data was loaded into a table with a defined schema.
3.  **Gemini API Integration:** The necessary libraries were installed, and the Gemini API was set up for use.
4.  **Prompt Engineering for Text-to-SQL:** A customized prompt was created for the employee database, including the schema definition and examples to guide the Gemini model in generating accurate SQL queries from natural language.
5.  **Natural Language Querying:** The `text2sql` function was utilized to convert natural language questions about the employee data into executable SQLite queries and display the results.

Through these steps, we have built an end-to-end solution that allows users to interact with the employee database using plain English, showcasing the power of Text-to-SQL for democratizing data access and analysis.